# Практикум Python


<img src="https://www.python.org/static/community_logos/python-logo-master-v3-TM.png" align="right" style="height: 200px;"/>

## Тема 11. Numpy. Оптимизация кода кода

### Numpy

Рассмотрим класс `Matrix`

In [1]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property                  #позволяет сделать метод класса его атрибутом
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

Реализуем умножение матриц

In [2]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [3]:
# функция генерации матриц
def setup(shape=(100, 100)):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    return X, Y

Замерим время работы функции умножения

In [37]:
%%timeit 
matrix_product(*setup())

529 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Теперь реализуем эту же функцию, но при помощи `numpy`

In [21]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [22]:
def setup_np(shape=(100, 100)):
    X = np.random.randint(-255, 255, size=shape)
    Y = np.random.randint(-255, 255, size=shape)
    return X, Y

In [36]:
%%timeit
np_matrix_product(*setup_np())

1.07 s ± 48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Никакого профита нет, выполнение даже замедлилось почти в 2 раза!

А теперь сделаем всё то же самое, но при помощи встроенной функции `dot()`

In [31]:
X, Y = setup_np()

In [35]:
%%timeit 
np.dot(X, Y)

941 µs ± 49.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Такой результат - отличная мотивация, чтобы изучить модуль `numpy`

### Array

In [38]:
a = np.array([4, 1.1, 2, 3, True], dtype=np.int32)

a[1] = 5

print(a.size, a.shape, a.dtype)
print(a)
print(type(a))

5 (5,) int32
[4 5 2 3 1]
<class 'numpy.ndarray'>


Можно менять `shape`

In [39]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

a.shape = (2, 2)
a

array([[4., 5.],
       [2., 3.]])

Массив имеет один тип. Если при создании передать разные типы, то он всё равно всё приведёт к одному типу.

In [44]:
b = np.array([4, 1.1, 2, 3, "string"])
b

array(['4', '1.1', '2', '3', 'string'], dtype='<U32')

Ко всем элементам можно получить доступ и манипулировать ими так же, как вы бы это делали с обычными списками:

In [45]:
b[:2], b[3]

(array(['4', '1.1'], dtype='<U32'), '3')

Массивы могут быть и многомерными. В отличии от списков можно использовать запятые в скобках

In [46]:
matrix = np.array([[1, 2, 3], [4, 5, 6]])

matrix[0, 2]

3

Срезы работают с многомерными массивами аналогично, как и с одномерными, применяя каждый срез, как фильтр для установленного измерения. Используйте ":" в измерении для указывания использования всех элементов этого измерения:

In [48]:
matrix[1, :], matrix[:, 2]

(array([4, 5, 6]), array([3, 6]))

Многомерный массив можно транспонировать

In [51]:
matrix.transpose() # или matrix.T 

array([[1, 4],
       [2, 5],
       [3, 6]])

Из массива можно сделать `list`

In [49]:
b.tolist()

['4', '1.1', '2', '3', 'string']

Возможно объединение массивов

In [77]:
c = np.arange(1, 10, 2)
d = np.arange(2, 11, 2)

np.concatenate((c, d)) # первый аргумент - кортеж объединяемых массивов

array([ 1,  3,  5,  7,  9,  2,  4,  6,  8, 10])

Многомерные массивы можно объединять по первым осям с помощью `hstack()` и по последним осям `vstack()`

In [84]:
a = np.array([[1, 2], [4, 5], [6, 7]])
b = np.array([[5, 6], [7, 8], [9, 10]])
a.shape, b.shape

((3, 2), (3, 2))

In [86]:
np.hstack((a, b))

array([[ 1,  2,  5,  6],
       [ 4,  5,  7,  8],
       [ 6,  7,  9, 10]])

In [85]:
np.vstack((a, b))

array([[ 1,  2],
       [ 4,  5],
       [ 6,  7],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10]])

#### UFunc

UFunc (Universal functions) - это функции, которые работают над массивом поэлементно.

In [52]:
a = range(4)
b = [value + 1 for value in a]
print(b)

[1, 2, 3, 4]


Аналогичный код, но реализованный при помощи `numpy`

In [54]:
a = np.arange(4)
b = a + 1
print(b)

[1 2 3 4]


Это работает для любых универсальных функций

Сложение двух array происходит поэлементно и при этом быстро:

In [56]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i + j for i, j in zip(a, b)]

1.76 s ± 151 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
a, b = np.array(a), np.array(b)
%timeit a + b

46 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Это работает быстро, так как всё итерирование протащено в компилируемую часть кода.

Есть и другие UFuncs, т.е. поэлементные функции. Например посмотрим, сколько займёт поэлементное возведение массива в квадрат:

In [58]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

4.19 ms ± 369 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

21.8 µs ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Есть много других UFuncs: арифметические операции, побитовые, сравнения, тригонометрические и т.д.

#### Aggregation

В `numpy` много встроенных агрегационных функций

**Function Name** | **NaN-safe Version** | **Description**
- | - | -
`np.sum` | `np.nansum` | Compute sum of elements
`np.prod` | `np.nanprod` | Compute product of elements
`np.mean` | `np.nanmean` | Compute mean of elements
`np.std` | `np.nanstd` | Compute standart deviation
`np.var` | `np.nanvar` | Compute variance
`np.min` | `np.nanmin` | Find minimum value
`np.max` | `np.nanmax` | Find maximum value
`np.argmin` | `np.nanargmin` | Find index of minimum value
`np.argmax` | `np.nanargmax` | Find index of maximum value
`np.median` | `np.nanmedian` | Compute median of elements
`np.percentile` | `np.nanpercentile` | Compute rank-based statistics of elements
`np.any` | - | Evaluate whether any elements are true
`np.all` | - | Evaluate whether all elements are true

P.S. `nan` - не число

In [60]:
print(float('nan'))
print(float('Inf') - float('Inf'))
print(np.inf - np.inf)

nan
nan
nan


In [61]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

18.2 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
np_data = np.array(data)
%timeit np.min(np_data)

725 µs ± 46.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Masking

Рассмотрим следующийфрагмент кода:

In [63]:
a = np.arange(8) ** 2
a[a > 8]

array([ 9, 16, 25, 36, 49], dtype=int32)

Это работает за счёт того, что numpy.array поддерживает индексацию по булевой маске:

In [64]:
a > 8

array([False, False, False,  True,  True,  True,  True,  True])

Так же можно использовать массив индексов:

In [66]:
a = np.arange(4) ** 2
print(a)
a[[3, 3, 2, 2, 0, 0, 1, 1]]

[0 1 4 9]


array([9, 9, 4, 4, 0, 0, 1, 1], dtype=int32)

#### Broadcasting

Или автоматическое приведение размерностей

Можно складывать массивы разных размеров там, где это имеет смысл:

In [68]:
a = np.arange(3)
b = np.array(((0, 0, 0), (10, 10, 10), (20, 20, 20), (30, 30, 30)))
a, b

(array([0, 1, 2]),
 array([[ 0,  0,  0],
        [10, 10, 10],
        [20, 20, 20],
        [30, 30, 30]]))

In [69]:
b + a

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

<img src="https://i.stack.imgur.com/JcKv1.png" width=900px/>

Но не всегда всё работает так, как хотелось бы:

In [70]:
X = np.arange(6).reshape((3, 2))
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [71]:
X + np.array([1, 2, 3])

ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

Мы, конечно, понимаем, что имелось в виду, но numpy не понял. Подскажем ему:

In [72]:
Y = np.array((1, 2, 3))[:, np.newaxis]  # переориентируем массив
print(Y.shape)
print(Y)

(3, 1)
[[1]
 [2]
 [3]]


In [73]:
X + Y

array([[1, 2],
       [4, 5],
       [7, 8]])

**Вывод:** циклы в python медленные. Стоит использовать встроенные функции `numpy`. Так и код понятнее и ускорение значительное.

**Bonus** в `numpy` также есть модуль `numpy.linalg`, который позволяет делать многие операции из линейной алгебры
https://numpy.org/doc/stable/reference/routines.linalg.html

## Cython

Ещё один способ ускорить ваш код

Компилятор Cython переводит программу, написанную на языках Python или Cython, в более эффективный код на С

Язык Cython является надстройкой Python, поддерживающей вызовы функций C и присвоение переменным типов данных языка C.

Посмотрим, как он работает. Создадим пару функций на чистом python:

In [87]:
def f(x):
    return x**2

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [88]:
%%timeit
integrate_f(0, 10, 1_000_000)

643 ms ± 48.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [89]:
!pip install Cython

In [93]:
%load_ext Cython

In [94]:
%%cython -a
def f(x):
    return x**2

def integrate_f2(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Cython пытается определить тип и в частности за счёт этого ускориться. Но желтым подсвечиваются те строки кода, где он не смог установить тип. Тут почти всё желтое.

In [95]:
%%timeit
integrate_f2(0, 10, 1000000)

372 ms ± 58.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Стало чуть быстрее. Но можно лучше.

Применим статическую типизацию к переменным и явно укажем тип возвращаемого функцией значения:

In [107]:
%%cython -a 
cdef double f2(double x):
    return x ** 2 

cdef double integrate_f4(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # в случае, когда i объевлено как int, этот цикл компилируется в чистый C код
        s += f2(a + i * dx)
    return s * dx

In [108]:
%%timeit
integrate_f4(0, 10, 1000000)

53.4 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Extra
[Numba](https://numba.pydata.org/) - классная библиотека для удобной jit-компиляции

## Оптимизация кода

Python-код можно оптимизировать не только с использованием дополнительных библиотек, нои собственными средствами

1. Используйте стандартные функции: многие из них реализованы на С, за счёт чего работаю быстрее, чем plain python. Такие функции покрывают большинство тривиальных вычислительных операций

In [114]:
%%timeit
sum_value = 0
for n in range(1, 11):
  sum_value += n

674 ns ± 18.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [115]:
%%timeit
sum(range(1, 11))

398 ns ± 21.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


2. Вместо цикла для создания списка эффективнее использовать генератор

In [116]:
%%timeit
even_num = []
for number in range(1, 1000):
    if number % 2 == 0:
        even_num.append(number)

126 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [118]:
%%timeit
even_num = [number for number in range(1, 1000) if number % 2 == 0]

81.6 µs ± 9.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


3. Иногда при переборе списка нужны и значения, и их индексы. Чтобы ускорить код используйте `enumerate()` для превращения списка в пары индекс-значение:

In [119]:
import random
random.seed(666)
a_short_list = [random.randint(0,500) for i in range(5)]

In [127]:
%%timeit
res = 0
for i in range(len(a_short_list)):
    res *= i + a_short_list[i]

936 ns ± 74.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [128]:
%%timeit
res = 0
for i, number in enumerate(a_short_list):
    res *= i + number

790 ns ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


4. Не следует явно указывать `== True` в условии `if`, достаточно указать имя проверяемой переменной. Это экономит ресурсы, которые использует «магическая» функция `__eq__` для сравнения значений.

In [142]:
string = 'pythoh is the best programming language'

In [143]:
%%timeit
if string == True:
    pass

47.2 ns ± 2.51 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [145]:
%%timeit
if string:
    pass

34.9 ns ± 0.98 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


5. "Дорогой" операцией являетсяобращение к элементу по индексу, поэтому стоит от неё избавляться по максимуму

Вспомним про реализованный в начале семинра класс `Matrix` и изменим реализацию умножения

In [146]:
# первая реализация

def matrix_product_1(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [147]:
# улучшенная реализация

def matrix_product_2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [148]:
%%timeit
matrix_product_1(*setup())

400 ms ± 24.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [149]:
%%timeit
matrix_product_2(*setup())

247 ms ± 9.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [150]:
# избавимся от еще одного обращения по индексу
def matrix_product_3(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

In [151]:
%%timeit
matrix_product_3(*setup())

214 ms ± 8.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Вывод:** если ваш код тормозит, не спешите использовать другие фреймворки. Часто можно ускорить в разы свой код небольшими манипуляциями.